In [1]:
import numpy as np
import pandas as pd

from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, InputLayer
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [2]:
# Reading the dataset
data_csv = "dataset.csv"
df = pd.read_csv(data_csv)
print('Dataset shape: ', df.shape)
print(df.dtypes)
df.head()

Dataset shape:  (1089, 7)
Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume       float64
dtype: object


,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-12-31,1196.400024,1212.400024,1182.000000,1201.900024,1201.900024,124.0
1,2014-01-02,1204.300049,1227.300049,1204.300049,1225.000000,1225.000000,209.0
2,2014-01-03,1221.699951,1239.000000,1221.699951,1238.400024,1238.400024,142.0
3,2014-01-06,1232.800049,1247.000000,1221.900024,1237.800049,1237.800049,127.0
4,2014-01-07,1239.300049,1242.400024,1226.300049,1229.400024,1229.400024,73.0


In [3]:
# Verifying null values and deleting name from dataset
null_columns=df.columns[df.isnull().any()]
print(df[df.isnull().any(axis=1)][null_columns].head())
# Drop the lines with null values
df = df.dropna()
# Drop Date column
# df.pop("Date")

print('Dataset shape: ', df.shape)

     Open  High  Low  Close  Adj Close  Volume
127   NaN   NaN  NaN    NaN        NaN     NaN
230   NaN   NaN  NaN    NaN        NaN     NaN
248   NaN   NaN  NaN    NaN        NaN     NaN
515   NaN   NaN  NaN    NaN        NaN     NaN
535   NaN   NaN  NaN    NaN        NaN     NaN
Dataset shape:  (1081, 7)


In [4]:
print("Minimum: {}\nMaximum: {}\nMean: {}\nMedian: {}\nSD: {}\nSkewness: {}\nKurtosis: {}".format(df["Low"].min(), df["High"].max(), 
df["Open"].mean(), df["Open"].median(), df["Open"].std(), df["Open"].skew(), df["Open"].kurtosis()))

Minimum: 1046.199951
Maximum: 1391.400024
Mean: 1239.9909346142476
Median: 1251.0
SD: 72.09937994027125
Skewness: -0.5455305158449637
Kurtosis: -0.3520037164214358


In [5]:
lastday_2017 = df.loc[df["Date"]=="2017-12-29"].index.values[0]
df = df["Close"].values

# Transforming the dataset to ln scale
df = np.log(df)

# # Split dataset into train and test
train_set = df[0:lastday_2017]
test_set = df[lastday_2017:]
print("Train: ", train_set.shape, "Test: ", test_set.shape)

Train:  (1007,) Test:  (74,)


In [6]:
lastday_2017

1007

In [7]:

# df   = df.reshape(len(df),1)
# scaler = MinMaxScaler(feature_range=(-1, 1))
# scaler.fit(df)
# df = scaler.transform(df).flatten()


## Labels of an LSTM network
Now for the labels of the LSTM network, not all the cases require a label matrix of 3 dimensions. The cases where this is required are on sequence to sequence problems, where the model is made to predict a sequence of timestamps of one or more features. However for our problem, the LSTM network needs to predict the next day gold price closing value, this way a matrix of 2 dimensions will suffice for this problem.

# Regression models

In [8]:
# FFNN class
class FFNN:
    def __init__(self, input_dim, scaler=None):
        self.scaler = scaler 
        optimizer = Adam()
        h_n = 3 if input_dim == 4 or input_dim == 6 else 5
        self.model = Sequential()
        self.model.add(Dense(h_n, input_dim=input_dim))
        self.model.add(Dense(1))
        self.model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=["accuracy", "mean_absolute_error"])
    
    def fit(self,x_train,y_train):
        if self.scaler:
            x_train = self.scaler.transform(x_train)
            y_train = self.scaler.transform(y_train)
        self.model.fit(x_train, y_train,
                        epochs=50,
                        batch_size=128,
                        verbose=0
                      )

    def predict(self, x_test):
        if self.scaler:
            x_test = self.scaler.transform(x_test)
            
        y_valid_pred = self.model.predict(x_test)
        
        if self.scaler:
            y_valid_pred = self.scaler.transform(y_valid_pred)

        return y_valid_pred.flatten()

In [9]:
# LSTM class
class PLSTM:
    def __init__(self, input_shape, model_type, scaler=None):
        self.scaler = scaler 
        optimizer = Adam()
        self.model = Sequential()
        self.h_n1 = 100 if model_type in [1,3,4] else 200
        return_seq = True if model_type>2 else False
        self.model.add(LSTM(units=self.h_n1, input_shape=(input_shape[1], 1), return_sequences=return_seq))
        if model_type>2:
            self.h_n2 = 50 if model_type == 3 else 100
            self.model.add(LSTM(units=self.h_n2))
            if type==4:
                self.model.add(Dense(32))
        self.model.add(Dense(1))
        self.model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=["accuracy", "mean_absolute_error"])
    
    def fit(self,x_train,y_train):
        if self.scaler:
            x_train = self.scaler.transform(x_train)
            y_train = self.scaler.transform(y_train)

        # reshape the entry as a 3D matrix with samples, timestamps and lastly features
        # instead of only samples and features as usual.
        x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
        history = self.model.fit(x_train, y_train,
                        epochs=50,
                        batch_size=128,
                        verbose=0)

    def predict(self, x_test):
        if self.scaler:
            x_test = self.scaler.transform(x_test)
            
        x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
        y_valid_pred = self.model.predict(x_test)
        
        if self.scaler:
            y_valid_pred = self.scaler.transform(y_valid_pred)

        return y_valid_pred.flatten()

In [10]:
# CNN-LSTM class
class CNNLSTM:
    def __init__(self, input_shape, model_type, scaler=None):
        self.scaler = scaler
        optimizer = Adam()
        self.model = Sequential()
        self.h_n1 = 100 if model_type == 1 else 200
        self.filter1 = 32 if model_type == 1 else 64
        self.filter2 = 64 if model_type == 1 else 128
        
        self.model.add(Conv1D(self.filter1, 2,activation='relu',
                       strides=1,
                       padding='same',
                       input_shape=(input_shape[1],
                                   1)))

        self.model.add(Conv1D(self.filter2, 2,
                   activation='relu',
                   strides=1,
                   padding='same',
                   input_shape=(input_shape[1],
                                1)))

        self.model.add(MaxPooling1D(pool_size=2, padding='valid'))
        self.model.add(LSTM(units=self.h_n1, input_shape=(input_shape[1],1)))
    
        if type==2:
            self.model.add(Dense(32))
        self.model.add(Dense(1))
        self.model.compile(loss="mean_squared_error", optimizer=optimizer)
    
    def fit(self,x_train,y_train):
        if self.scaler:
            x_train = self.scaler.transform(x_train)
            y_train = self.scaler.transform(y_train)
            
        # reshape the entry as a 3D matrix with samples, timestamps and lastly features
        # instead of only samples and features as usual.
        x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
        
        self.model.fit(x_train, y_train,
                        epochs=50,
                        batch_size=128,
                        verbose=0
                      )

    def predict(self, x_test):
        if self.scaler:
            x_test = self.scaler.transform(x_test)
            
        x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
        y_valid_pred = self.model.predict(x_test)
        
        if self.scaler:
            y_valid_pred = self.scaler.transform(y_valid_pred)

        return y_valid_pred.flatten()

In [11]:
def create_models(entry_shape, scaler=None):
    # SVR
    svr = SVR(kernel='rbf', C=1, tol=1e-3)

    # FFNN
    ffnn = FFNN(entry_shape[1], scaler)

    # LSTM1
    lstm1 = PLSTM(entry_shape, 1, scaler)

    # LSTM2
    lstm2 = PLSTM(entry_shape, 2, scaler)

    # LSTM3
    lstm3 = PLSTM(entry_shape, 3, scaler)

    # LSTM4
    lstm4 = PLSTM(entry_shape, 4, scaler)

    # CNN-LSTM1
    cnnlstm1 = CNNLSTM(entry_shape, 1, scaler)

    # CNN-LSTM2
    cnnlstm2 = CNNLSTM(entry_shape, 2, scaler)

    labels = ["SVR", "FFNN", "LSTM1", "LSTM2", "LSTM3", "LSTM4", "CNN-LSTM1", "CNN-LSTM2"]
    models = [svr, ffnn, lstm1, lstm2, lstm3, lstm4, cnnlstm1, cnnlstm2]

    return labels, models

## Rolling window approach
The paper states that in order to predict the next day gold price, the model uses the $n$ past days gold prince, where $n$ stands for the time horizon used. Thus to generate a dataset with this specifications, we will use a rolling window algorithm to generate a window of features to a window of labels ( with in this case is equal to 1). This rolling window procedure works as follows:

Features: $[n1, n2, n3, n4, n5]$ -> Label $[n6]$

In [12]:
def rolling_window_mtx(x, window_size):
        """Compute all overlapping (rolling) observation windows over a vector 
            and return a matrix

        Args:
            x           : observation vector that is supposed to be split into
                          overlapping windows
            window_size : the target window size

        Returns:

            Window matrix with all windows as rows. That is, if n_windows is the
            number of windows, the result has dimensions:

            (n_windows, window_size)

        """
        if window_size < 1:
            raise ValueError("`window_size` must be at least 1.")
        if window_size > x.shape[-1]:
            raise ValueError("`window_size` is too long.")

        shape = x.shape[:-1] + (x.shape[-1] - window_size + 1, window_size)
        strides = x.strides + (x.strides[-1],)

        return np.lib.stride_tricks.as_strided(x, shape=shape, strides=strides)

In [13]:
def generate_feat_labels_per_horizon(time_horizon, df, verbose=False):

    # Get the feature and label to the prediction task 
    feature_mtx = rolling_window_mtx(df, time_horizon)[:-1]
    label_mtx   = rolling_window_mtx(df[time_horizon:], 1)
    index_mtx   = rolling_window_mtx(np.arange(len(df)), time_horizon)

    if verbose:
        # Now we have a set of windows of the real coordinate
        # Lets take a look in one window
        print(f"\n One feature window: \n {feature_mtx[0]}")
        print(f"\n One label window: \n {label_mtx[0]}")
        print(f"\n Original dataset: \n {df[0:5]}")

    # For the classification task (if the gold values goes up or down)
    # We need to get a window of size 2, and then calculate the difference
    # If positive, the gold value went up.
    class_label_mtx = rolling_window_mtx(df[time_horizon-1:], 2)
    func = lambda x: True if x > 0 else False
    class_func = np.vectorize(func)
    class_label_mtx = class_func(np.diff(class_label_mtx).flatten()).reshape(len(class_label_mtx),1)
   
    if verbose:
    
        print(f"\n One window of class label (If tomorrow price is larger than today's price): \n {class_label_mtx[0]}")
    
    return feature_mtx, label_mtx, class_label_mtx

In [14]:
label_index_mtx = rolling_window_mtx(np.arange(len(df))[4:], 1)
index_mtx   = rolling_window_mtx(np.arange(len(df)), 4)[:-1]

In [15]:
train_idx = lastday_2017 - 4
index_mtx[train_idx], label_index_mtx[train_idx]

(array([1003, 1004, 1005, 1006]), array([1007]))

## Normalizing dataset

In [16]:
df = df.reshape(len(df),1)
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(df)
df = df.flatten()
df

array([7.09165894, 7.11069612, 7.12157552, ..., 7.18629566, 7.18258009,
       7.1856143 ])

In [17]:
entries = [4, 6, 9]
models = []
labels = []
for entry in entries:
    #Creating dataset
    feature_mtx, label_mtx, class_label_mtx = generate_feat_labels_per_horizon(entry, df)
    scaler_input = MinMaxScaler(feature_range=(-1, 1))
    scaler_input.fit(feature_mtx)
    scaler_output = MinMaxScaler(feature_range=(-1, 1))
    scaler_output.fit(label_mtx)
    train_idx = lastday_2017 - entry
    train_x = feature_mtx[:train_idx]
    train_y = label_mtx[:train_idx]
    tmp_labels, tmp_models = create_models(train_x.shape)#, scaler)
    for i in range(len(tmp_models)):
        print(tmp_labels[i])
        tmp_models[i].fit(train_x, train_y)
    models.append(tmp_models)
    labels.append(tmp_labels)

SVR
FFNN


/home/rodrigo/home-hd/Anaconda/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LSTM1
LSTM2
LSTM3
LSTM4
CNN-LSTM1
CNN-LSTM2
SVR
FFNN


/home/rodrigo/home-hd/Anaconda/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LSTM1
LSTM2
LSTM3
LSTM4
CNN-LSTM1
CNN-LSTM2
SVR
FFNN


/home/rodrigo/home-hd/Anaconda/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LSTM1
LSTM2
LSTM3
LSTM4
CNN-LSTM1
CNN-LSTM2


In [18]:
models = np.ravel(models)
labels = np.ravel(labels)
print(len(models))

24


In [19]:
# Testing models
index_model = 0
for entry in entries:
    print ("# Entries: ", entry)
    feature_mtx, label_mtx, class_label_mtx = generate_feat_labels_per_horizon(entry, df)
    scaler_input = MinMaxScaler(feature_range=(-1, 1))
    scaler_input.fit(feature_mtx)
    train_idx = lastday_2017 - entry
    test_x = feature_mtx[train_idx:]
    test_y = label_mtx[train_idx:]
    for i in range(index_model, index_model+8):
        test_y_estimative = models[i].predict(test_x)
        print("Classifier type: ", labels[i], "\nMAE = ", mean_absolute_error(test_y, test_y_estimative),"\nRMSE = ", mean_squared_error(test_y, test_y_estimative, squared=True), "\n")
    index_model += 8

# Entries:  4
Classifier type:  SVR 
MAE =  0.07038084116745039 
RMSE =  0.005027129598584784 

Classifier type:  FFNN 
MAE =  0.010736886356252785 
RMSE =  0.00018829635080284761 

Classifier type:  LSTM1 
MAE =  0.06185956868897483 
RMSE =  0.00391713536605355 

Classifier type:  LSTM2 
MAE =  0.0582892955731466 
RMSE =  0.00348758578373553 

Classifier type:  LSTM3 
MAE =  0.07429580396218138 
RMSE =  0.005614268973018944 

Classifier type:  LSTM4 
MAE =  0.07069251953876567 
RMSE =  0.00508967579619689 

Classifier type:  CNN-LSTM1 
MAE =  0.05501622732553192 
RMSE =  0.003099392904652124 

Classifier type:  CNN-LSTM2 
MAE =  0.05701815725304855 
RMSE =  0.0033230862410679836 

# Entries:  6
Classifier type:  SVR 
MAE =  0.06938287345347131 
RMSE =  0.00489374749799132 

Classifier type:  FFNN 
MAE =  0.011763308394609607 
RMSE =  0.00021399763801736955 

Classifier type:  LSTM1 
MAE =  0.0742012097799865 
RMSE =  0.005599159859005228 

Classifier type:  LSTM2 
MAE =  0.06970908156